Universidade do Vale do Itajaí<br>
Escola Politécnica<br>
Processamento Digital de Sinais: Imagens

# Código de exemplo - Convolução para Detecção de Bordas - Filtro Canny

### Tutoriais da OpenCV

- https://docs.opencv.org/master/d9/df8/tutorial_root.html
- https://www.geeksforgeeks.org/opencv-python-tutorial/

Vamos conhecer um detector de bordas muito mais avançado, o Canny. Este detector é baseado na aplicação de duas convoluções, em sentidos diferentes, seguida de uma limiarização e esqueletonização. Os resultados obtidos com ele são considerados os melhores que você pode conseguir com algoritmos que não apelam para heurísticas complexas, como o método de detecção de curvas salientes de Sha'ashua.

# Detector de borda Canny


O detector de bordas Canny é um operador de detecção de bordas que usa um algoritmo de vários estágios para detectar uma ampla variedade de bordas em imagens. Foi desenvolvido por John F. Canny em 1986. Canny também produziu uma teoria computacional de detecção de bordas explicando por que a técnica funciona.

O algoritmo de detecção de borda do Processo de Canny pode ser dividido em 5 etapas diferentes:

- Aplique filtro gaussiano para suavizar a imagem e remover o ruído
- Encontre os gradientes de intensidade da imagem
- Aplique supressão não máxima para se livrar de respostas espúrias à detecção de bordas
- Aplicar limite (threshold) duplo para determinar arestas potenciais
- Rastrear borda por histerese: Finalize a detecção de bordas suprimindo todas as outras bordas que são fracas e não conectadas a bordas fortes
    
 * Fonte: https://en.wikipedia.org/wiki/Canny_edge_detector
 * Excelente implementação detalhada: [TowardsDataScience::Canny Edge Detection Step by Step in Python by Sofiane Sahir](https://towardsdatascience.com/canny-edge-detection-step-by-step-in-python-computer-vision-b49c3a2d8123)

Referências
----------

1. Canny, J., A Computational Approach To Edge Detection, IEEE Transactions on Pattern Analysis and Machine Intelligence, 8(6):679–698, 1986.

2. R. Deriche, Using Canny's criteria to derive a recursively implemented optimal edge detector, Int. J. Computer Vision, Vol. 1, pp. 167–187, April 1987.




Obs.: Não usaremos o blur acima porque as implementações do Canny no OpenCV e SciKit já o possuem.

In [13]:
from scipy import ndimage
import numpy as np
import cv2
from scipy import ndimage
from matplotlib import pyplot as plt
from ipywidgets import interact_manual

# Teste se seu notebook está rodando no Google Colab
# Você usará isso ao escolher entre interagir via ipywidgets ou não. 
try:
  import google.colab
  _ON_COLAB = True
except:
  _ON_COLAB = False

print('Running on Google Colab = ', _ON_COLAB)

Running on Google Colab =  False


In [16]:
import skimage

sources = [
     'skimage.data.astronaut()',
     'skimage.data.camera()',
     'skimage.data.checkerboard()',
     'skimage.data.chelsea()',
     'skimage.data.clock()',
     'skimage.data.coffee()',
     'skimage.data.coins()',
     'skimage.data.horse()',
     'skimage.data.hubble_deep_field()',
     'skimage.data.immunohistochemistry()',
     'skimage.data.lbp_frontal_face_cascade_filename()',
     'skimage.data.lfw_subset()',
     'skimage.data.logo()',
     'skimage.data.microaneurysms()',
     'skimage.data.moon()',
     'skimage.data.page()',
     'skimage.data.text()',
     'skimage.data.retina()',
     'skimage.data.rocket()',
     'skimage.data.stereo_motorcycle()']

def to_gray(image):
     # If the image is colored, make a grayscale copy
    if image.shape[-1] == 3:           
        b,g,r = cv2.split(image)       # get b,g,r
        rgb_img = cv2.merge([r,g,b])     # switch it to rgb
        gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = image
    return gray_img

## SciKit

O detector de borda Canny que estamos vendo aqui é este: http://scikit-image.org/docs/dev/auto_examples/edges/plot_canny.html. A documentação da função está aqui: https://github.com/scikit-image/scikit-image/blob/master/skimage/feature/_canny.py

Esta implementação do algoritmo de detecção de bordas Canny funciona apenas em imagens em tons de cinza!

In [17]:
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm


def cannySciKit(source, sigma, low_threshold, high_threshold, use_quantiles):
    # Localização de linha usando a Transformada Probabilística de Hough
    image = eval(source)
    gray_img = to_gray(image)

    edges = canny(gray_img, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold, use_quantiles=use_quantiles)

    # Generating figure 2
    fig, axes = plt.subplots(1, 2, figsize=(15, 7), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(image)
    ax[0].set_title('Input image')

    ax[1].imshow(edges, cmap=cm.gray)
    ax[1].set_title('Canny edges')

    for a in ax:
        a.set_axis_off()

    plt.tight_layout()
    plt.show()
    
if _ON_COLAB:
    # Google Colab:
    cannySciKit(source='skimage.data.coffee()', sigma=4.5, low_threshold=1, high_threshold=25, use_quantiles=False)
else:
    # Jupyter Notebook:
    interact_manual(cannySciKit, source=sources, sigma=(0.2, 11.0), low_threshold=1, high_threshold=25, use_quantiles=False)

interactive(children=(Dropdown(description='source', options=('skimage.data.astronaut()', 'skimage.data.camera…

## OpenCV

Esta implementação do algoritmo de detecção de bordas Canny funciona em imagens coloridas!

In [18]:
from skimage import data
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import cv2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

def cannyOpenCV(source,threshold1,threshold2,apertureSize,L2gradient):
    image = eval(source)

    # Crie uma imagem de borda com a implementação inteligente do OpenCV. Empregue imagens coloridas, se desejar.
    edges = cv2.Canny(image, threshold1=threshold1, threshold2=threshold2, apertureSize=apertureSize, L2gradient=L2gradient)
    # apertureSize é o tamanho da abertura para o operador de detecção de borda Sobel

    # Escreva o resultado no disco
    #cv2.imwrite('cannyEdges3.jpg',edges)
    
    # Crie uma linha de dois gráficos para mostrar a imagem e as bordas
    fig, axes = plt.subplots(1, 2, figsize=(15, 7), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow(image)
    ax[0].set_title('Input image')

    ax[1].imshow(edges, cmap=cm.gray)
    ax[1].set_title('Canny edges')

    for a in ax:
        a.set_axis_off()

    plt.tight_layout()
    plt.show()


if _ON_COLAB:
    # Google Colab:
    cannyOpenCV(source='skimage.data.coffee()',threshold1=100,threshold2=500,apertureSize=3,L2gradient=True)
else:
    # Jupyter Notebook:
    interact_manual(cannyOpenCV, source=sources, threshold1=100,threshold2=500,apertureSize=3,L2gradient=True)

interactive(children=(Dropdown(description='source', options=('skimage.data.astronaut()', 'skimage.data.camera…